In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pickle as pkl
import subprocess
import napari
from magicgui import magicgui

from multiplex_pipeline.utils import get_workstation_path, change_to_wsl_path, get_package_path, load_config
from multiplex_pipeline.im_utils import prepare_rgb_image, get_org_im_shape
from multiplex_pipeline.roi_utils import get_refined_rectangles, xywh_to_corners

from qtpy.QtWidgets import QFileDialog
import pandas as pd
from shapely.geometry import Polygon
from multiplex_pipeline.roi_utils import prepare_poly_df_for_saving, xywh_to_corners, get_visual_rectangles

In [61]:
# load config
# this step ensures that all starting parameters are loaded before they are used
config = load_config(namespace = globals())

Using workstation DESKTOP-S71S74E

IM_LEVEL: 6
MIN_AREA: 2000
MAX_AREA: 10000
MIN_IOU: 0.8
MIN_ST: 0.9
MIN_INT: 15
FRAME: 4


In [ ]:
# pathway to the image for segmetnation

im_path = r'R:\CellDive\BLCA-1B\BLCA-1B_Final\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif'

# output pathway
# where to save corrected cores coordinates
save_path = r'R:\Wayne\BLCA\BLCA-1B_Analysis\cores.csv'

os.makedirs(os.path.dirname(save_path), exist_ok = True)

## Display image and options

In [5]:
# get an image for display

# chnge the level paramter if necessary
#IM_LEVEL = 6

# get the original image shape
org_im_shape = get_org_im_shape(im_path)

# prepare the image for display
im_rgb = prepare_rgb_image(im_path, req_level=IM_LEVEL)
im = im_rgb[:, :, 0]
im_rgb.shape

(1072, 558, 3)

In [ ]:


def redo_cores_layer(viewer,data=[],shape_type='polygon'):

    if 'cores' in viewer.layers:
        viewer.layers.remove('cores')

    viewer.add_shapes(
    data,       
    shape_type=shape_type,  
    edge_color='green', 
    face_color='transparent',  
    edge_width=2,       
    name = 'cores'
)

def redo_bbox_layer(viewer,data=[],text=[]):

    if 'bounding_boxes' in viewer.layers:
        viewer.layers.remove('bounding_boxes')

    viewer.add_shapes(
    data,       
    shape_type='rectangle',  
    edge_color='red', 
    face_color='transparent',  
    edge_width=2,       
    name = 'bounding_boxes',
    text = {'string': text,'size':12,'color':'red','anchor':'upper_left'}
)    


def save_rois(viewer, org_im_shape, req_level, save_path = None):
    '''
    '''
    if 'cores' in viewer.layers:

        # get the saving path if not provided
        if save_path is None:
            # open dialog for getting a dir to save csv file
            save_path = QFileDialog.getSaveFileName(filter = 'CSV file (*.csv)')[0]

        # get the polygon data
        poly_data = viewer.layers['cores'].data
        poly_types = viewer.layers['cores'].shape_type

        # prepare df for saving
        df = prepare_poly_df_for_saving(poly_data, poly_types, req_level, org_im_shape)

        # save the rois
        df.to_pickle(save_path.replace('.csv','.pkl'))
        df.to_csv(save_path, index = False)

        # prepare the cores visual for saving
        rect_list = get_visual_rectangles(df, req_level)
        poly_list = [(x/(2**(req_level))).astype('int') for x in df.polygon_vertices.to_list()]

        # change the visualization
        redo_cores_layer(viewer,poly_list,shape_type = df.poly_type.to_list())
        redo_bbox_layer(viewer,rect_list,df['core_name'].tolist())

        viewer.status = f'Cores saved to {save_path}'

    else:
        viewer.status = 'No layer called "cores" found!'

In [58]:
viewer = napari.Viewer()
viewer.add_image(im)

# add a red rectangle to frame the image
frame_rect = xywh_to_corners([0,0,im.shape[1],im.shape[0]], frame = 0)
viewer.add_shapes(frame_rect, edge_color='white', face_color = 'transparent', shape_type='rectangle', edge_width=2, name = 'frame')

redo_cores_layer(viewer,[])

# add a save button
@magicgui(auto_call=False, call_button="Save Cores")  
def save_button(viewer: napari.Viewer):
    save_rois(viewer, org_im_shape = org_im_shape, req_level = IM_LEVEL, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(save_button, area='left')

## Get suggestions for core detection from the SAM2 model

In [69]:
MAX_AREA = 14000

In [67]:
# Change the default if needed
# IM_LEVEL: 6

# Path to the Python interpreter in the target Conda environment
sam_env = get_workstation_path(key="sam_env")

# Path to your script
script_path = change_to_wsl_path(os.path.join(get_package_path('multiplex_pipeline'),'suggest_cores.py'))
im_wsl_path = change_to_wsl_path(im_path)

# Define the script arguments
output_path_suggested = os.path.join(os.getcwd(), 'suggested_cores.csv')

optional_args = ['--output', change_to_wsl_path(output_path_suggested)]

# Combine the command
command = ['wsl', sam_env, '-u', script_path, im_wsl_path, str(IM_LEVEL)] + optional_args

# Execute the script
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,  # Pipe the standard output
    stderr=subprocess.PIPE,  # Pipe the standard error
    text=True                # Capture output as text
)

# Display output in real time
while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output, end="")  # Print each line as it becomes available

# Print any remaining errors
errors = process.stderr.read()
if errors:
    print(f"ERROR: {errors}")

No model path provided. Using default model path: /mnt/d/data_analysis/2024_bladder/sam2
Input image: /mnt/r/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif
Results will be saved to: /mnt/d/multiplex_pipeline/notebooks/suggested_cores.csv

Preparing RGB image for segmentation...
Segmenting image. It should take around 1 min. Started at 2025-06-05 15:19:08...
Saving masks...


In [70]:
# load the masks

masks = pkl.load(open(output_path_suggested, 'rb'))
print(f'Initial number of masks: {len(masks)}')

# refine masks
rect_list = get_refined_rectangles(masks, im = im_rgb[:,:,0], frame = FRAME, min_area = MIN_AREA, max_area = MAX_AREA, min_iou = MIN_IOU, min_stability = MIN_ST, min_int = MIN_INT)
print('Number of refined masks:', len(rect_list))

# add to napari
redo_cores_layer(viewer,rect_list,shape_type='rectangle')

Initial number of masks: 101
Number of refined masks: 49
